In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import glob
import matplotlib.pyplot as plt 
from collections import deque 
from tensorflow.keras import layers
import time 
import scipy.io as sio

from vehicle_model_variant import Environment 
from cell_model import CellModel 
from driver_MDP import Driver_MDP 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
# env = Environment(cell_model, drving_cycle, battery_path, motor_path, 10)
driver = Driver_MDP(0.02)

num_states = 4

In [3]:
class OUActionNoise: 
    def __init__(self, mean, std_deviation, theta=0.15, dt=1e-2, x_initial=None): 
        self.theta = theta 
        self.mean = mean 
        self.std_dev = std_deviation 
        self.dt = dt 
        self.x_initial = x_initial 
        self.reset() 
        
    def reset(self): 
        if self.x_initial is not None: 
            self.x_prev = self.x_initial 
        else: 
            self.x_prev = 0 
            
    def __call__(self): 
        x = (
             self.x_prev + self.theta * (self.mean - self.x_prev) * self.dt 
            + self.std_dev * np.sqrt(self.dt) * np.random.normal() 
        )
        self.x_prev = x 
        return x 

In [4]:
class Buffer: 
    def __init__(self, buffer_capacity=100000, batch_size=64):      
        self.buffer_capacity = buffer_capacity 
        self.batch_size = batch_size 
        self.buffer_counter = 0 
        
        self.state_buffer = np.zeros((self.buffer_capacity, num_states))
        self.action_buffer = np.zeros((self.buffer_capacity, 1))
        self.reward_buffer = np.zeros((self.buffer_capacity, 1))
        self.next_state_buffer = np.zeros((self.buffer_capacity, num_states))
        
    def record(self, obs_tuple):
        index = self.buffer_counter % self.buffer_capacity 

        self.state_buffer[index] = obs_tuple[0]
        self.action_buffer[index] = obs_tuple[1]
        self.reward_buffer[index] = obs_tuple[2]
        self.next_state_buffer[index] = obs_tuple[3]
        
        self.buffer_counter += 1 
        
    def learn(self): 
        record_range = min(self.buffer_counter, self.buffer_capacity)
        batch_indices = np.random.choice(record_range, self.batch_size)

        state_batch = tf.convert_to_tensor(self.state_buffer[batch_indices])
        action_batch = tf.convert_to_tensor(self.action_buffer[batch_indices])
        reward_batch = tf.convert_to_tensor(self.reward_buffer[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        next_state_batch = tf.convert_to_tensor(self.next_state_buffer[batch_indices])
        
        with tf.GradientTape() as tape: 
            target_actions = target_actor(next_state_batch)
            y = reward_batch + gamma * target_critic([next_state_batch, target_actions])
            critic_value = critic_model([state_batch, action_batch])
            critic_loss = tf.math.reduce_mean(tf.square(y - critic_value)) 
        critic_grad = tape.gradient(critic_loss, critic_model.trainable_variables) 
        critic_optimizer.apply_gradients(
            zip(critic_grad, critic_model.trainable_variables)
        )
        
        with tf.GradientTape() as tape: 
            actions = actor_model(state_batch)
            critic_value = critic_model([state_batch, actions])
            actor_loss = - tf.math.reduce_mean(critic_value)
        actor_grad = tape.gradient(actor_loss, actor_model.trainable_variables) 
        actor_optimizer.apply_gradients(
            zip(actor_grad, actor_model.trainable_variables)
        )
        

In [5]:
def update_target(tau): 
    new_weights = [] 
    target_variables = target_critic.weights
    for i, variable in enumerate(critic_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_critic.set_weights(new_weights)
    
    new_weights = [] 
    target_variables = target_actor.weights
    for i, variable in enumerate(actor_model.weights): 
        new_weights.append(target_variables[i] * (1 - tau) + tau * variable)
    target_actor.set_weights(new_weights)
    

In [6]:
def get_actor(): 
    last_init = tf.random_uniform_initializer(minval=-0.003, maxval=0.003)
    
    inputs = layers.Input(shape=(num_states))
    inputs_batchnorm = layers.BatchNormalization()(inputs)
    
    out = layers.Dense(512, activation="relu")(inputs_batchnorm)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1, activation="sigmoid", 
                          kernel_initializer=last_init)(out)
    model = tf.keras.Model(inputs, outputs)
    return model

In [7]:
def get_critic(): 
    state_input = layers.Input(shape=(num_states))
    state_input_batchnorm = layers.BatchNormalization()(state_input)
    
    state_out = layers.Dense(16, activation="relu")(state_input_batchnorm)
#     state_out = layers.BatchNormalization()(state_out)
    state_out = layers.Dense(32, activation="relu")(state_out)
#     state_out = layers.BatchNormalization()(state_out)
    
    action_input = layers.Input(shape=(1))
    action_out = layers.Dense(32, activation="relu")(action_input)
#     action_out = layers.BatchNormalization()(action_out)
    
    concat = layers.Concatenate()([state_out, action_out]) 
    
    out = layers.Dense(512, activation="relu")(concat)
#     out = layers.BatchNormalization()(out)
    out = layers.Dense(512, activation="relu")(out)
#     out = layers.BatchNormalization()(out)
    outputs = layers.Dense(1)(out)
    
    model = tf.keras.Model([state_input, action_input], outputs)
    return model 
    

In [8]:
def policy(state, noise_object): 
    j_min = state[0][2].numpy()
    j_max = state[0][3].numpy()
    sampled_action = tf.squeeze(actor_model(state)) 
    noise = noise_object()
    sampled_action = sampled_action.numpy() + noise 
    legal_action = sampled_action * j_max 
    legal_action = np.clip(legal_action, j_min, j_max)
#     print(j_min, j_max, legal_action, noise)
    return legal_action 
    

In [9]:
def policy_epsilon_greedy(state, eps): 
    j_min = state[0][-2].numpy()
    j_max = state[0][-1].numpy()

    if random.random() < eps: 
        a = random.randint(0, 9)
        return np.linspace(j_min, j_max, 10)[a]
    else: 
        sampled_action = tf.squeeze(actor_model(state)).numpy()  
        legal_action = sampled_action * j_max 
        legal_action = np.clip(legal_action, j_min, j_max)
        return legal_action

In [10]:
std_dev = 0.2 
ou_noise = OUActionNoise(mean=0, std_deviation=0.2)

critic_lr = 0.0005 
actor_lr = 0.00025 
critic_optimizer = tf.keras.optimizers.Adam(critic_lr)
actor_optimizer = tf.keras.optimizers.Adam(actor_lr)

total_episodes = 200 
gamma = 0.95 
tau = 0.001 

MAX_EPSILON = 1.0 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
DELAY_TRAINING = 5000 

In [11]:
def initialization(): 
    actor_model = get_actor() 
    critic_model = get_critic() 
    target_actor = get_actor() 
    target_critic = get_critic() 
    target_actor.set_weights(actor_model.get_weights())
    target_critic.set_weights(critic_model.get_weights())
#     actor_model.load_weights("./DDPG1_trial1/actor_model_checkpoint")
#     critic_model.load_weights("./DDPG1_trial1/critic_model_checkpoint")
#     target_actor.load_weights("./DDPG1_trial1/target_actor_checkpoint")
#     target_critic.load_weights("./DDPG1_trial1/target_critic_checkpoint")
    
    buffer = Buffer(500000, BATCH_SIZE)
    return actor_model, critic_model, target_actor, target_critic, buffer

In [12]:
def save_weights(actor_model, critic_model, target_actor, target_critic, root): 
    if not os.path.exists(root): 
        os.makedirs(root)
        
    actor_model.save_weights("./{}/actor_model.h5".format(root))
    critic_model.save_weights("./{}/critic_model.h5".format(root))
    target_actor.save_weights("./{}/target_actor.h5".format(root))
    target_critic.save_weights("./{}/target_critic.h5".format(root))
    print("model is saved..")

In [13]:
def initialization_env(driving_path, reward_factor):
    env = Environment(cell_model, driving_path, battery_path, motor_path, reward_factor)
    return env 

In [14]:
def test_agent(actor_model, reward_factor):
#     test_cycle = driver.get_cycle() 
    test_cycle_path = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
    test_cycle = sio.loadmat(test_cycle_path)
    test_cycle = test_cycle["sch_cycle"][:, 1]
    env = initialization_env(test_cycle, reward_factor)
    
    total_reward = 0
    state = env.reset() 
    while True: 
        tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
        action = policy_epsilon_greedy(tf_state, -1)
        next_state, reward, done = env.step(action)
        
        state = next_state 
        total_reward += reward 
        
        if done: 
            break 
        
    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
    
    print("******************* Test is start *****************")
#     print(test_cycle)
    print('Total reward: {}'.format(total_reward), 
          "SOC: {:.4f}".format(env.SOC), 
          "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
          "Fuel Consumption: {:.4f}".format(env.fuel_consumption))
    print("******************* Test is done *****************")
    print("")
    plt.subplot(1, 2, 1)
    plt.plot(test_cycle)
    plt.subplot(1, 2, 2)
    plt.plot(env.history["Action"])
    plt.show() 
    return env.history  
    

In [15]:
# print(env.version)

# num_trials = 1
reward_factors = [1, 2, 3]
results_dict = {} 
driving_cycle_path = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
driving_cycle = sio.loadmat(driving_cycle_path)
driving_cycle = driving_cycle["sch_cycle"][:, 1]

for reward_factor in reward_factors: 
    print("")
    print("Trial {}".format(reward_factor))
    print("")
    
    actor_model, critic_model, target_actor, target_critic, buffer = initialization()
    
    eps = MAX_EPSILON 
    steps = 0
    
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    episode_test_history = [] 
    episode_num_test = [] 
    for ep in range(total_episodes): 
#         driving_cycle = driver.get_cycle() 
        env = initialization_env(driving_cycle, reward_factor)
        
        start = time.time() 
        state = env.reset() 
        episodic_reward = 0 

        while True: 
            tf_state = tf.expand_dims(tf.convert_to_tensor(state), 0)
            action = policy_epsilon_greedy(tf_state, eps)
    #         print(action)
            next_state, reward, done = env.step(action)
            if done: 
                next_state = [0] * num_states 

            buffer.record((state, action, reward, next_state))
            episodic_reward += reward 

            if steps > DELAY_TRAINING: 
                buffer.learn() 
                update_target(tau)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * (steps
                                                                        -DELAY_TRAINING))

            steps += 1

            if done: 
                break 

            state = next_state 

        elapsed_time = time.time() - start 
        print("elapsed_time: {:.3f}".format(elapsed_time))
        episode_rewards.append(episodic_reward) 
        episode_SOCs.append(env.SOC)
        episode_FCs.append(env.fuel_consumption) 

    #     print("Episode * {} * Avg Reward is ==> {}".format(ep, avg_reward))
        SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
        print(
              'Episode: {}'.format(ep + 1),
              "Exploration P: {:.4f}".format(eps),
              'Total reward: {}'.format(episodic_reward), 
              "SOC: {:.4f}".format(env.SOC), 
              "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
              "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
        )
        print("")
        
        if (ep + 1) % 10 == 0: 
#             history = test_agent(actor_model, reward_factor)
            history = env.history 
            episode_test_history.append(history) 
            episode_num_test.append(ep + 1)
            
#         if (ep + 1) % 200 == 0:             
    root = "DDPG_cycleOne_reward_factor{}".format(reward_factor)
    save_weights(actor_model, critic_model, target_actor, target_critic, root)
            
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs, 
        "test_history": episode_test_history, 
        "test_episode_num": episode_num_test, 
    }


Trial 1

maximum steps, simulation is done ... 
elapsed_time: 9.183
Episode: 1 Exploration P: 1.0000 Total reward: -669.6301542086956 SOC: 1.0000 Cumulative_SOC_deviation: 481.0124 Fuel Consumption: 188.6177

maximum steps, simulation is done ... 
elapsed_time: 9.215
Episode: 2 Exploration P: 1.0000 Total reward: -676.4870294699607 SOC: 1.0000 Cumulative_SOC_deviation: 485.6016 Fuel Consumption: 190.8855

maximum steps, simulation is done ... 
elapsed_time: 9.735
Episode: 3 Exploration P: 1.0000 Total reward: -679.2840680106785 SOC: 1.0000 Cumulative_SOC_deviation: 486.4813 Fuel Consumption: 192.8027


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author

maximum steps, simulation is done ... 
elapsed_time: 85.941
Episode: 28 Exploration P: 0.5172 Total reward: -252.02888495345096 SOC: 0.7988 Cumulative_SOC_deviation: 110.7555 Fuel Consumption: 141.2734

maximum steps, simulation is done ... 
elapsed_time: 88.245
Episode: 29 Exploration P: 0.5034 Total reward: -228.27196514526017 SOC: 0.8119 Cumulative_SOC_deviation: 85.7822 Fuel Consumption: 142.4898

maximum steps, simulation is done ... 
elapsed_time: 86.816
Episode: 30 Exploration P: 0.4901 Total reward: -207.32358707736768 SOC: 0.7098 Cumulative_SOC_deviation: 72.5441 Fuel Consumption: 134.7794

maximum steps, simulation is done ... 
elapsed_time: 88.213
Episode: 31 Exploration P: 0.4771 Total reward: -208.99212990416424 SOC: 0.7334 Cumulative_SOC_deviation: 72.1923 Fuel Consumption: 136.7998

maximum steps, simulation is done ... 
elapsed_time: 86.177
Episode: 32 Exploration P: 0.4644 Total reward: -197.18436552198955 SOC: 0.5708 Cumulative_SOC_deviation: 72.9912 Fuel Consumption:

C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment_essential_modelChange\DDPG_rewardfactor_comparison_final\vehicle_model_variant.py:251: RuntimeWarning: invalid value encountered in double_scalars
  del_i = (1 / (2 * r_cha)) * (v_cha - (v_cha ** 2 - 4 * r_cha * p_bat) ** (0.5)) * (p_bat < 0) + (1 / (


maximum steps, simulation is done ... 
elapsed_time: 79.754
Episode: 44 Exploration P: 0.3368 Total reward: -593.2858489213535 SOC: 0.0289 Cumulative_SOC_deviation: 505.9538 Fuel Consumption: 87.3321

maximum steps, simulation is done ... 
elapsed_time: 80.379
Episode: 45 Exploration P: 0.3280 Total reward: -447.37486371392083 SOC: 0.1661 Cumulative_SOC_deviation: 352.2500 Fuel Consumption: 95.1249

maximum steps, simulation is done ... 
elapsed_time: 79.680
Episode: 46 Exploration P: 0.3194 Total reward: -560.9083022402615 SOC: 0.0608 Cumulative_SOC_deviation: 472.0404 Fuel Consumption: 88.8680

maximum steps, simulation is done ... 
elapsed_time: 79.935
Episode: 47 Exploration P: 0.3110 Total reward: -559.2155137571144 SOC: 0.0409 Cumulative_SOC_deviation: 471.7597 Fuel Consumption: 87.4558

battery power is 7946.590307366354(+) but condition is not avail
elapsed_time: 65.825
Episode: 48 Exploration P: 0.3043 Total reward: -1466.5895096886507 SOC: -0.0014 Cumulative_SOC_deviation: 39

C:\Users\ValueCreatorSong\Desktop\Academic\graduate_paper\degradation_model\experiment_essential_modelChange\DDPG_rewardfactor_comparison_final\vehicle_model_variant.py:252: RuntimeWarning: invalid value encountered in double_scalars
  2 * r_dis)) * (v_dis - (v_dis ** 2 - 4 * r_dis * p_bat) ** (0.5)) * (p_bat >= 0)


...

maximum steps, simulation is done ... 
elapsed_time: 83.540
Episode: 19 Exploration P: 0.6594 Total reward: -1133.202968875912 SOC: 0.9837 Cumulative_SOC_deviation: 327.0732 Fuel Consumption: 151.9834

maximum steps, simulation is done ... 
elapsed_time: 83.357
Episode: 20 Exploration P: 0.6418 Total reward: -1131.7996355666503 SOC: 0.9995 Cumulative_SOC_deviation: 325.3513 Fuel Consumption: 155.7458

maximum steps, simulation is done ... 
elapsed_time: 83.489
Episode: 21 Exploration P: 0.6247 Total reward: -1212.2083063504758 SOC: 0.9939 Cumulative_SOC_deviation: 353.5389 Fuel Consumption: 151.5917

maximum steps, simulation is done ... 
elapsed_time: 83.521
Episode: 22 Exploration P: 0.6080 Total reward: -904.5444357247202 SOC: 0.9858 Cumulative_SOC_deviation: 250.4448 Fuel Consumption: 153.2100

maximum steps, simulation is done ... 
elapsed_time: 83.640
Episode: 23 Exploration P: 0.5918 Total reward: -926.5329887585237 SOC: 0.9737 Cumulative_SOC_deviation: 258.2004 Fuel Consumption

battery power is 11598.447586310172(+) but condition is not avail
elapsed_time: 79.345
Episode: 59 Exploration P: 0.2472 Total reward: -2458.8340884053005 SOC: -0.0006 Cumulative_SOC_deviation: 461.0007 Fuel Consumption: 77.6345

maximum steps, simulation is done ... 
elapsed_time: 85.505
Episode: 60 Exploration P: 0.2408 Total reward: -1133.3108825057423 SOC: 0.5598 Cumulative_SOC_deviation: 335.5717 Fuel Consumption: 126.5959

maximum steps, simulation is done ... 
elapsed_time: 85.454
Episode: 61 Exploration P: 0.2345 Total reward: -314.9756045485569 SOC: 0.5244 Cumulative_SOC_deviation: 67.5972 Fuel Consumption: 112.1839

maximum steps, simulation is done ... 
elapsed_time: 85.726
Episode: 62 Exploration P: 0.2284 Total reward: -390.2480702275489 SOC: 0.5155 Cumulative_SOC_deviation: 91.9882 Fuel Consumption: 114.2835

maximum steps, simulation is done ... 
elapsed_time: 85.455
Episode: 63 Exploration P: 0.2225 Total reward: -563.2901131748979 SOC: 0.5108 Cumulative_SOC_deviation: 

battery power is 6345.029147455788(+) but condition is not avail
elapsed_time: 57.707
Episode: 99 Exploration P: 0.0934 Total reward: -1900.217844921307 SOC: -0.0011 Cumulative_SOC_deviation: 282.8135 Fuel Consumption: 53.5816

battery power is 7957.703445167204(+) but condition is not avail
elapsed_time: 56.551
Episode: 100 Exploration P: 0.0919 Total reward: -2030.3451697482465 SOC: -0.0002 Cumulative_SOC_deviation: 321.0167 Fuel Consumption: 69.0964

battery power is 5157.377688440195(+) but condition is not avail
elapsed_time: 52.020
Episode: 101 Exploration P: 0.0905 Total reward: -2167.660678700922 SOC: -0.0001 Cumulative_SOC_deviation: 345.3480 Fuel Consumption: 133.5106

battery power is 4556.04963138524(+) but condition is not avail
elapsed_time: 63.827
Episode: 102 Exploration P: 0.0889 Total reward: -2257.1571034563085 SOC: -0.0002 Cumulative_SOC_deviation: 400.8236 Fuel Consumption: 56.4879

battery power is 10880.247543512363(+) but condition is not avail
elapsed_time: 45.

maximum steps, simulation is done ... 
elapsed_time: 85.091
Episode: 136 Exploration P: 0.0488 Total reward: -1750.0215604879945 SOC: 0.2003 Cumulative_SOC_deviation: 551.7881 Fuel Consumption: 94.6574

maximum steps, simulation is done ... 
elapsed_time: 85.088
Episode: 137 Exploration P: 0.0478 Total reward: -1154.1286350623254 SOC: 0.1934 Cumulative_SOC_deviation: 355.6129 Fuel Consumption: 87.2899

maximum steps, simulation is done ... 
elapsed_time: 84.466
Episode: 138 Exploration P: 0.0468 Total reward: -1217.0541835653262 SOC: 0.3509 Cumulative_SOC_deviation: 372.7651 Fuel Consumption: 98.7590

maximum steps, simulation is done ... 
elapsed_time: 84.326
Episode: 139 Exploration P: 0.0458 Total reward: -894.2997956627557 SOC: 0.4041 Cumulative_SOC_deviation: 264.7323 Fuel Consumption: 100.1030

maximum steps, simulation is done ... 
elapsed_time: 84.362
Episode: 140 Exploration P: 0.0448 Total reward: -861.7396369459348 SOC: 0.3704 Cumulative_SOC_deviation: 254.7663 Fuel Consumpt

maximum steps, simulation is done ... 
elapsed_time: 85.266
Episode: 176 Exploration P: 0.0242 Total reward: -1019.3583632189956 SOC: 0.3774 Cumulative_SOC_deviation: 305.4539 Fuel Consumption: 102.9968

maximum steps, simulation is done ... 
elapsed_time: 84.979
Episode: 177 Exploration P: 0.0239 Total reward: -1461.0677624353839 SOC: 0.2679 Cumulative_SOC_deviation: 455.3662 Fuel Consumption: 94.9691

maximum steps, simulation is done ... 
elapsed_time: 85.332
Episode: 178 Exploration P: 0.0235 Total reward: -1231.9035820133827 SOC: 0.3457 Cumulative_SOC_deviation: 376.6801 Fuel Consumption: 101.8633

maximum steps, simulation is done ... 
elapsed_time: 85.074
Episode: 179 Exploration P: 0.0231 Total reward: -1213.110175225524 SOC: 0.2742 Cumulative_SOC_deviation: 374.2194 Fuel Consumption: 90.4520

maximum steps, simulation is done ... 
elapsed_time: 85.223
Episode: 180 Exploration P: 0.0228 Total reward: -923.2535899601295 SOC: 0.3998 Cumulative_SOC_deviation: 274.9729 Fuel Consump

In [16]:
with open("DDPG_cycleOne_1to3.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [17]:
# results_dict